In [1]:
import pandas as pd
import polars as pl
import json

### Authors

In [8]:
df = pd.read_json('goodreads_book_authors.json', lines=True)

In [19]:
df = df.drop(columns='text_reviews_count')
df

,average_rating,author_id,name,ratings_count
0,3.98,604031,Ronald J. Fields,49
1,4.08,626222,Anita Diamant,546796
2,3.92,10333,Barbara Hambly,122118
3,3.68,9212,Jennifer Weiner,888522
4,3.82,149918,Nigel Pennick,1740
...,...,...,...,...
829524,4.36,197551,Patty Furbush,11
829525,4.33,3988103,Jim Schlinkman,6
829526,4.00,13464507,Rich Jolly,18
829527,3.31,7427847,sr@ mwrGn,13


### Interactions

In [5]:
interact_path = "C:/Users/Usuario/Datos/goodreads_interactions.csv"
new_path = "C:/Users/Usuario/Datos/interactions_filtered.csv"

# Lee el archivo CSV
df = pl.read_csv(interact_path)

# Filtra las filas donde is_read == 0
df_filtrado = df.filter(pl.col('is_read') != 0)

# Guarda el resultado en un nuevo archivo CSV
df_filtrado.write_csv(new_path)

print("Filtrado completado. El archivo filtrado se ha guardado como 'archivo_filtrado.csv'.")


Filtrado completado. El archivo filtrado se ha guardado como 'archivo_filtrado.csv'.


In [9]:
final_path = "C:/Users/Usuario/Datos/interactions_sample.csv"
user_counts = df_filtrado.groupby('user_id').agg(pl.count('is_read').alias('count'))

# Paso 2: Seleccionar una muestra aleatoria de 20,000 user_id únicos
sampled_user_ids = user_counts.sample(n=20000, with_replacement=False).select('user_id')

# Paso 3: Filtrar el DataFrame original usando estos user_id seleccionados
df_sampled = df_filtrado.join(sampled_user_ids, on='user_id', how='inner')

df_sampled.write_csv(final_path)

# Verificar el resultado
print(df_sampled.shape)
print(df_sampled.head())

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10080\4251902593.py:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  user_counts = df_filtrado.groupby('user_id').agg(pl.count('is_read').alias('count'))


(2699934, 5)
shape: (5, 5)
┌─────────┬─────────┬─────────┬────────┬─────────────┐
│ user_id ┆ book_id ┆ is_read ┆ rating ┆ is_reviewed │
│ ---     ┆ ---     ┆ ---     ┆ ---    ┆ ---         │
│ i64     ┆ i64     ┆ i64     ┆ i64    ┆ i64         │
╞═════════╪═════════╪═════════╪════════╪═════════════╡
│ 102     ┆ 670     ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 998     ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 996     ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 12971   ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 43      ┆ 1       ┆ 5      ┆ 0           │
└─────────┴─────────┴─────────┴────────┴─────────────┘


In [3]:
df_samples = pl.read_csv('/Users/ucemarc/Downloads/interactions_sample.csv')

# Supongamos que df_filtrado ya está cargado
# df_filtrado = pl.read_csv('ruta/al/archivo_filtrado.csv')

# Obtener los valores únicos de la columna 'book_id'
unique_book_ids = df_samples.select('book_id').unique()

# Convertir a una lista
book_id_list = unique_book_ids.to_series().to_list()
book_id_list = [str(book_id) for book_id in book_id_list]

# Imprimir el número de book_id únicos y una muestra de los primeros 10
print(f"Número de book_id únicos: {len(book_id_list)}")
print(f"Primeros 10 book_id únicos: {book_id_list[:10]}")

Número de book_id únicos: 535192
Primeros 10 book_id únicos: ['740496', '247657', '2052071', '86651', '178953', '1051167', '1423667', '1543601', '1281454', '228498']


In [4]:
def preprocess_chunk(chunk):
    # Nos quedamos con las columnas "book_id", "authors", "similar_books", "publication_year", "title"
    chunk = chunk[["book_id", "authors", "similar_books", "publication_year", "title"]]
    return chunk

In [5]:
books_path = "/Users/ucemarc/Downloads/goodreads_books.json"
new_path = "/Users/ucemarc/Downloads/books_sample.csv"

# Create SchemaDefinition
schema = {
    'book_id': str,
    'authors': list[str],
    'similar_books': list[str],
    'publication_year': str,
    'title': str
}

# Lee el archivo CSV
df_list = []
with open(books_path, 'r') as file:
    for i, line in enumerate(file):
        # Cargar la línea JSON en un DataFrame de Polars
        json_data = json.loads(line.strip())
        # Get list of author_ids
        authors = json_data.get('authors', [])
        author_ids = [author.get('author_id') for author in authors]
        json_data['authors'] = author_ids
        try:
            json_df = pl.DataFrame([json_data], schema=schema)
        except Exception as e:
            print(f"Error en la línea {i}: {e}")
            print(json_data)
            print(author_ids)
            break
        df_filtrado = json_df.filter(pl.col('book_id').is_in(book_id_list))
        df_filtrado = preprocess_chunk(df_filtrado)
        # Añadir el DataFrame a la lista
        df_list.append(df_filtrado)

# Concatenar todos los DataFrames en uno solo
df_concatenado = pl.concat(df_list, how='vertical')
df_pandas = df_concatenado.to_pandas()
# Guarda el resultado en un nuevo archivo CSV
df_pandas.to_csv(new_path, index=False)

print("Filtrado completado. El archivo filtrado se ha guardado como 'archivo_filtrado.csv'.")

In [36]:
df_list

[shape: (0, 5)
 ┌─────────┬───────────┬───────────────┬──────────────────┬───────┐
 │ book_id ┆ authors   ┆ similar_books ┆ publication_year ┆ title │
 │ ---     ┆ ---       ┆ ---           ┆ ---              ┆ ---   │
 │ str     ┆ list[str] ┆ list[str]     ┆ str              ┆ str   │
 ╞═════════╪═══════════╪═══════════════╪══════════════════╪═══════╡
 └─────────┴───────────┴───────────────┴──────────────────┴───────┘,
 shape: (0, 5)
 ┌─────────┬───────────┬───────────────┬──────────────────┬───────┐
 │ book_id ┆ authors   ┆ similar_books ┆ publication_year ┆ title │
 │ ---     ┆ ---       ┆ ---           ┆ ---              ┆ ---   │
 │ str     ┆ list[str] ┆ list[str]     ┆ str              ┆ str   │
 ╞═════════╪═══════════╪═══════════════╪══════════════════╪═══════╡
 └─────────┴───────────┴───────────────┴──────────────────┴───────┘,
 shape: (0, 5)
 ┌─────────┬───────────┬───────────────┬──────────────────┬───────┐
 │ book_id ┆ authors   ┆ similar_books ┆ publication_year ┆ title │
 

In [ ]:
import polars as pl

# Define la ruta al archivo que contiene las filas de JSON
ruta_archivo_json = 'ruta/al/archivo.json'

# Lee el archivo línea por línea y convierte cada línea JSON en un DataFrame de Polars
df_list = []

with open(ruta_archivo_json, 'r') as file:
    for line in file:
        # Cargar la línea JSON en un DataFrame de Polars
        json_df = pl.read_json(line)
        # Añadir el DataFrame a la lista
        df_list.append(json_df)

# Concatenar todos los DataFrames en uno solo
df_concatenado = pl.concat(df_list, how='vertical')

# Imprimir información del DataFrame concatenado
print(df_concatenado.shape)
print(df_concatenado.head())

# Si deseas guardar el DataFrame concatenado en un archivo CSV
df_concatenado.write_csv('ruta/al/archivo_concatenado.csv')

print("Lectura y concatenación de JSON completadas. El archivo CSV se ha guardado como 'archivo_concatenado.csv'.")